In [23]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import json
import pickle
import jsonpickle
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [24]:
#df = pd.read_pickle('/home/jm/Documents/testingStuff/my_file.pkl')
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = False)
df = pd.read_sql_table("sawi_tweets_historical", con = engine)

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


In [25]:
df.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,0,2017-09-24 23:54:27,2,None,None,None,0,AmanteTech,Would next #Apple #iOS update cause your exist...,912102964881367040,...,2133,635,632,869739471570325504,"Sydney, New South Wales",AmanteTech,6314,Pacific Time (US & Canada),0,0
1,1,2017-09-24 23:30:33,0,None,None,None,84,supermeg500,RT @MegsSkunkey: iPhone 8 giveaway!!\nhttps://...,912096951532847104,...,61,361,4512,819370348420284417,,Giveaways~,402,None,0,1
2,2,2017-09-24 22:45:06,0,None,None,None,1860,botlamaya,RT @Sudathchamin: iPhone X new price?Ur left k...,912085514282553344,...,10,617,274,941255144,Colombo,Bot Lamaya [beta],51626,New Delhi,0,1
3,3,2017-09-24 22:45:06,0,None,None,None,1860,hikma_5000,RT @Sudathchamin: iPhone X new price?Ur left k...,912085512776798213,...,1388,143,113,791115735200182272,"Mansfield, TX",Hikma🌹🌹🌹,5432,Pacific Time (US & Canada),0,1
4,4,2017-09-24 22:36:56,0,None,None,None,4,Joey2031,RT @BoinkHunter: Beautiful device. \n#iPhoneX ...,912083456552783873,...,853,66,75,705859071044423680,,Joey,646,None,0,1


In [26]:
df.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,45266.000000,45266.000000,45266.000000,4.526600e+04,45266.000000,4.526600e+04,45266.000000,4.526600e+04,4.526600e+04,45266.000000,45266.000000
mean,49.195864,0.912915,3879.166328,9.146185e+17,8344.810454,9.786278e+03,1840.460346,1.827840e+17,4.709686e+04,0.014514,0.512062
std,28.846285,39.827144,13091.820219,6.425545e+14,23357.029302,1.148350e+05,11001.654680,3.433272e+17,1.319984e+05,0.119599,0.499860
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.210000e+02,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.145397e+17,106.000000,9.800000e+01,130.000000,2.828020e+08,1.526250e+03,0.000000,0.000000
50%,49.000000,0.000000,2.000000,9.145858e+17,1238.000000,3.720000e+02,356.000000,1.630756e+09,8.583500e+03,0.000000,1.000000
75%,74.000000,0.000000,751.000000,9.153291e+17,6862.750000,1.303000e+03,977.000000,4.195543e+09,3.622075e+04,0.000000,1.000000
max,99.000000,8016.000000,142956.000000,9.153658e+17,717217.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [27]:
df.count()

index                      45266
created_at                 45266
favorite_count             45266
in_reply_to_screen_name     3913
in_reply_to_status_id       3276
in_reply_to_user_id         3913
retweet_count              45266
screen_name                45266
text                       45266
tweet_id                   45266
tweet_location                87
user_created_at            45266
user_description           45266
user_favorites_count       45266
user_followers_count       45266
user_friends_count         45266
user_id                    45266
user_location              45266
user_name                  45266
user_statuses_count        45266
user_time_zone             27351
user_verified              45266
RT                         45266
dtype: int64

In [28]:
dfs = df.drop_duplicates(subset = 'tweet_id', inplace=False)

In [29]:
dfs.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,44629.000000,44629.000000,44629.000000,4.462900e+04,44629.000000,4.462900e+04,44629.000000,4.462900e+04,4.462900e+04,44629.000000,44629.000000
mean,49.235027,0.921531,3933.924533,9.146478e+17,8429.744740,9.884793e+03,1844.093594,1.798917e+17,4.741416e+04,0.014542,0.511708
std,28.844561,40.110049,13176.832575,5.903429e+14,23485.935539,1.156411e+05,11060.342246,3.410354e+17,1.321803e+05,0.119712,0.499869
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.210000e+02,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.145416e+17,112.000000,1.000000e+02,132.000000,2.832035e+08,1.569000e+03,0.000000,0.000000
50%,49.000000,0.000000,2.000000,9.145874e+17,1264.000000,3.750000e+02,358.000000,1.617632e+09,8.751000e+03,0.000000,1.000000
75%,74.000000,0.000000,781.000000,9.153297e+17,6990.000000,1.303000e+03,978.000000,4.085374e+09,3.658700e+04,0.000000,1.000000
max,99.000000,8016.000000,142956.000000,9.153658e+17,717217.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [30]:
dfs.count()

index                      44629
created_at                 44629
favorite_count             44629
in_reply_to_screen_name     3912
in_reply_to_status_id       3276
in_reply_to_user_id         3912
retweet_count              44629
screen_name                44629
text                       44629
tweet_id                   44629
tweet_location                82
user_created_at            44629
user_description           44629
user_favorites_count       44629
user_followers_count       44629
user_friends_count         44629
user_id                    44629
user_location              44629
user_name                  44629
user_statuses_count        44629
user_time_zone             27033
user_verified              44629
RT                         44629
dtype: int64

In [31]:
cols = list(dfs)
#cols - https://stackoverflow.com/a/25122293

Delete duplicates which have all same tweet text

In [32]:
dfd = dfs.drop_duplicates(subset = 'text', inplace=False)

In [33]:
dfd.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,26070.000000,26070.000000,26070.000000,2.607000e+04,26070.000000,2.607000e+04,26070.000000,2.607000e+04,2.607000e+04,26070.000000,26070.000000
mean,48.990871,1.554622,210.658765,9.145924e+17,6813.620061,1.509797e+04,2377.811201,1.774463e+17,6.143335e+04,0.022363,0.171270
std,28.812405,52.406674,2902.534675,5.909488e+14,21633.345276,1.445748e+05,13733.327003,3.397761e+17,1.589958e+05,0.147863,0.376752
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.172000e+03,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.142440e+17,38.000000,1.230000e+02,121.000000,2.116498e+08,2.106000e+03,0.000000,0.000000
50%,49.000000,0.000000,0.000000,9.145809e+17,647.000000,4.730000e+02,386.000000,1.352177e+09,1.132350e+04,0.000000,0.000000
75%,74.000000,0.000000,0.000000,9.146327e+17,4853.750000,1.862500e+03,1195.000000,3.784276e+09,4.735225e+04,0.000000,0.000000
max,99.000000,8016.000000,142109.000000,9.153658e+17,595418.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [34]:
# drop Nan if it is in specific column
ddp = dfd.dropna(axis=0, subset=['in_reply_to_screen_name'], how='any')

In [35]:
ddp.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
309,2,2017-09-28 21:02:49,0,VICE,9.13461566338716e17,23818581,0,bdnic11,@VICE #apple cant even supply the #iphoneX. Th...,913509324617482240,...,34906,266,732,2395236192,"San Jose, CA",RoBobby,3165,Pacific Time (US & Canada),0,0
1136,24,2017-09-20 08:24:39,1,HerrBeutel,9.10419298421821e17,42705767,1,H69tties_,@HerrBeutel IOS 11 BUG AND FIX.\nhttps://t.co/...,910419419758600193,...,203,24,14,724108259976724480,,Hotties Shoutout,21,None,0,0
1137,25,2017-09-20 08:24:15,0,imtheonlyjuan13,9.10419108822368e17,156617913,0,H69tties_,@imtheonlyjuan13 IOS 11 BUG AND FIX.\nhttps://...,910419321666416640,...,203,24,14,724108259976724480,,Hotties Shoutout,21,None,0,0
1138,26,2017-09-20 08:24:04,0,ConceptInfoway,9.10419155811045e17,28091952,0,H69tties_,@ConceptInfoway IOS 11 BUG AND FIX.\nhttps://t...,910419273473957888,...,203,24,14,724108259976724480,,Hotties Shoutout,21,None,0,0
1467,13,2017-09-30 21:46:08,0,buy_AnotherCoat,9.1423803711418e17,3382182143,0,DannyRezin57,@buy_AnotherCoat @XXL @CQuinn37 Wiz has an iPh...,914244999092555776,...,2720,121,260,2337417858,"Chicago, IL",Danny Ramirez,1214,Central Time (US & Canada),0,0


In [36]:
ddp.to_csv("twitter_dataset_v2.csv", float_format='{:f}'.format)